In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Import data

In [2]:
df = pd.read_csv("../../raw_data/master_df_with_webscraping.csv")

In [6]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3916 entries, 0 to 3915
Data columns (total 101 columns):
 #    Column                         Non-Null Count  Dtype  
---   ------                         --------------  -----  
 0    fee_cleaned                    3916 non-null   float64
 1    age                            3916 non-null   int64  
 2    height_cm                      3916 non-null   int64  
 3    weight_kg                      3916 non-null   int64  
 4    nationality                    3916 non-null   object 
 5    club_name                      3891 non-null   object 
 6    league_name                    3891 non-null   object 
 7    league_rank                    3891 non-null   float64
 8    overall                        3916 non-null   int64  
 9    potential                      3916 non-null   int64  
 10   wage_eur                       3916 non-null   int64  
 11   player_positions               3916 non-null   object 
 12   preferred_foot                 3

In [78]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [81]:
df_train.to_csv("../../raw_data/master_data_train.csv")
df_test.to_csv("../../raw_data/master_data_test.csv")

In [8]:
numericals = ["int64", "float64"]

In [10]:
X = df.select_dtypes(numericals).drop(columns="fee_cleaned")

In [12]:
y = df.fee_cleaned

# Feature selection

In [20]:
correlations = df.select_dtypes(numericals).corr().fee_cleaned

In [34]:
correlations.sort_values(ascending=False)[1:11]

release_clause_eur          0.561710
potential                   0.459864
wage_eur                    0.427816
gk_handling                 0.375701
overall                     0.365972
international_reputation    0.317760
movement_reactions          0.309499
gk_reflexes                 0.304895
gk_positioning              0.301471
gk_kicking                  0.299445
Name: fee_cleaned, dtype: float64

In [70]:
top_5_corr_num_features = list(correlations.sort_values(ascending=False)[1:6].index)

In [71]:
X_selection = X[top_5_corr_num_features]

In [72]:
X_selection

,release_clause_eur,potential,wage_eur,gk_handling,overall
0,NaN,50,2000,NaN,41
1,NaN,78,80000,NaN,78
2,NaN,79,90000,NaN,79
3,NaN,73,9000,NaN,69
4,NaN,78,20000,NaN,70
...,...,...,...,...,...
3911,81800000.0,87,77000,NaN,84
3912,9800000.0,83,14000,NaN,72
3913,731000.0,70,2000,NaN,61
3914,450000.0,81,500,NaN,56


# Train model 1: top 5 correlated numerical features

In [73]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_selection, y, test_size=0.2)

In [74]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config; set_config(display='diagram')
from sklearn.linear_model import LinearRegression, Ridge

In [75]:
pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
    ("scaler", MinMaxScaler()),
    ("estimator", LinearRegression())
])
pipe

Pipeline(steps=[('imputer', SimpleImputer(fill_value=0, strategy='constant')),
                ('scaler', MinMaxScaler()), ('estimator', LinearRegression())])

In [76]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe, X_train, y_train, cv=5, scoring='r2').mean()

0.26148294570824016